In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
data_fp = "./dataset_v2.csv"
df = pd.read_csv(data_fp, index_col=0)
df.head()

Date   Adj Close       Close        High         Low        Open  \
0  2021-12-30  178.199997  178.199997  180.570007  178.089996  179.470001   
1  2021-12-31  177.570007  177.570007  179.229996  177.259995  178.089996   
2  2022-01-03  182.009995  182.009995  182.880005  177.710007  177.830002   
3  2022-01-04  179.699997  179.699997  182.940002  179.119995  182.630005   
4  2022-01-05  174.919998  174.919998  180.169998  174.639999  179.610001   

      Volume  52_Week_High  52_Week_Low       SMA50  ...  Correlated_ADX  \
0   59773000    177.423706   177.423706  158.876484  ...       16.269091   
1   64062300    177.423706   177.423706  159.437839  ...       16.041491   
2  104487900    179.076584   179.076584  160.082240  ...       16.117357   
3   99310400    179.076584   179.076584  160.696709  ...       15.771742   
4   94537600    179.076584   179.076584  161.218101  ...       15.611580   

   Correlated_NIE  Correlated_EOS  Correlated_QQQX  Correlated_BST  \
0       29.549999       18.268600        23.806171       49.131287   
1       29.410000       18.072416        23.489511       47.450821   
2       28.740000       18.025335        23.286530       47.391159   
3       28.010000       17.491713        22.628855       45.352722   
4       27.799999       17.444633        22.222883       44.815769   

   Correlated_CSQ  Correlated_MSFT  Correlated_ETG  Correlated_ETY  \
0       18.802198       318.269989       18.040215       11.676160   
1       18.573509       304.799988       17.810352       11.535393   
2       18.553623       310.200012       17.754875       11.488467   
3       18.136017       302.649994       17.493299       11.332055   
4       17.966988       303.329987       17.517090       11.230388   

   Correlated_EXG  
0       10.532752  
1       10.493303  
2       10.503164  
3       10.335507  
4       10.325646  

[5 rows x 77 columns]

In [3]:
df.columns

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       '52_Week_High', '52_Week_Low', 'SMA50', 'SMA200', 'EMA50', 'EMA200',
       'MACD', 'MACD_S', 'MACD_H', 'ADX', 'RSI50', 'RSI200', '%K', '%D',
       '%Change', 'OBV', 'CMF', 'VWAP', 'Bollinger_UP', 'Bollinger_DOWN',
       'Bollinger_WIDTH', 'ATR', 'Volatility', 'Sentiment_Change',
       'Sentiment_MSPR', 'Historical_RatingScore',
       'Historical_RatingDetailsDCFScore', 'Historical_RatingDetailsROEScore',
       'Historical_RatingDetailsROAScore', 'Historical_RatingDetailsDEScore',
       'Historical_RatingDetailsPEScore', 'Historical_RatingDetailsPBScore',
       'IncomeStatement_Revenue', 'IncomeStatement_CostOfRevenue',
       'IncomeStatement_GrossProfit', 'IncomeStatement_GrossProfitRatio',
       'IncomeStatement_ResearchAndDevelopmentExpenses',
       'IncomeStatement_GeneralAndAdministrativeExpenses',
       'IncomeStatement_SellingAndMarketingExpenses',
       'IncomeStatement_SellingGeneralAndAdm

Below are the features that we "care about" there are other features that come from the income statement etc. We want to make up for the lack of training data with a large number of features (even though some will just get droppped).

```
'Date' - Date
'Adj Close' - Close price w.r.t. dividends & stock splits (basically, we use this one)
'Close' - drop
'High' - high price of stock
'Low' - low price of stock
'Open', - drop (this is close shifted by 1)
'Volume' - the amount of the stock that was traded
'52_Week_High' - high price over 52 weeks
'52_Week_Low' - low price over 52 weeks
'SMA50' - 50 day simple moving average
'SMA200' - 200 day simple moving average
'EMA50' - 50 day exponential moving average
'EMA200' - 200 day exponential moving average
'MACD' - Moving Average Convergence Divergence (difference between short-term and long-term EMAs, typically 12-day and 26-day)
'MACD_S' - MACD Signal line (9-day EMA of the MACD)
'MACD_H' - MACD Histogram (difference between MACD and its Signal line)
'ADX' - Average Directional Index (measures trend strength, values range from 0 to 100)
'RSI50' - 50-day Relative Strength Index (measures overbought/oversold conditions, ranges from 0 to 100)
'RSI200' - 200-day Relative Strength Index (measures overbought/oversold conditions over a longer period)
'%K' - Stochastic Oscillator %K (measures the level of the close relative to the high-low range over a specified period)
'%D' - Stochastic Oscillator %D (3-day moving average of %K, used to generate buy/sell signals)
'%Change' - Percentage change in price over a specified period (usually 1 day or multiple)
'OBV' - On-Balance Volume (a volume-based indicator that adds volume on up days and subtracts volume on down days to gauge buying/selling pressure)
'CMF' - Chaikin Money Flow (measures the flow of money into or out of a stock over a specified period, typically 21 days)
'VWAP' - Volume Weighted Average Price (average price of a security, weighted by volume, during a trading day)
'Bollinger_UP' - Upper Bollinger Band (a line two standard deviations above the 20-day simple moving average)
'Bollinger_DOWN' - Lower Bollinger Band (a line two standard deviations below the 20-day simple moving average)
'Bollinger_WIDTH' - Bollinger Band Width (the difference between the upper and lower Bollinger Bands, used to measure volatility)
'ATR' - Average True Range (measures market volatility, calculated by averaging the true range over a set number of periods)
'Volatility' - A measure of the price fluctuations of a security, often represented as standard deviation of returns
'Sentiment_Change' - Change in sentiment score over a specified period (positive or negative sentiment shift based on news, tweets, etc.)
'Sentiment_MSPR' - Sentiment Mean-Standardized Price Ratio (average sentiment score relative to price action)
'Historical_RatingScore' - Composite rating score based on various fundamental metrics
'Historical_RatingDetailsDCFScore' - Discounted Cash Flow (DCF) based rating score for the stock
'Historical_RatingDetailsROEScore' - Return on Equity (ROE) based rating score for the stock
'Historical_RatingDetailsROAScore' - Return on Assets (ROA) based rating score for the stock
'Historical_RatingDetailsDEScore' - Debt-to-Equity (D/E) ratio based rating score for the stock
'Historical_RatingDetailsPEScore' - Price-to-Earnings (P/E) ratio based rating score for the stock
'Historical_RatingDetailsPBScore' - Price-to-Book (P/B) ratio based rating score for the stock
```

In [4]:
# for simplicity rename close
df = df.drop(["Open", "Close"], axis=1).rename({"Adj Close": "Close"}, axis=1)
df = df.drop([col for col in df.columns if col.startswith("IncomeStatement")], axis=1)

Decide what we are trying to predict.

In [5]:
# let's predict the movement of a stock n days in the future
n_days = 14
df[f"Future_Close_{n_days}"] = df["Close"].shift(-n_days)
df = df.dropna()

Calculate Percent Change

In [6]:
df["%Close"] = df["Close"].pct_change()
df[f"Future_%Close_{n_days}"] = df["Close"].pct_change(periods=n_days).shift(-n_days)
df = df.dropna()

Scale our features

In [8]:
scaler = MinMaxScaler()

cols_to_normalize = [col for col in df.columns if df[col].dtype != 'object' and df[col].abs().max() > 1 and '%' not in col]
normalized_data = scaler.fit_transform(df[cols_to_normalize])
df[cols_to_normalize] = normalized_data

Now, let's make this a classification problem. 

Keep in mind when training our model, we may want to adjust weight adjustment as being wrong in -10% < x < 10%  is much better than the stock increasing when we say that the stock is going to decrease by 50%

In [9]:
df.head()

Date     Close      High       Low    Volume  52_Week_High  \
1  2021-12-31  0.478598  0.470126  0.487422  0.135810      0.080465   
2  2022-01-03  0.519035  0.503472  0.491554  0.273018      0.107192   
3  2022-01-04  0.497996  0.504020  0.504499  0.255445      0.107192   
4  2022-01-05  0.454463  0.478714  0.463368  0.239246      0.107192   
5  2022-01-06  0.427869  0.434223  0.435824  0.247277      0.107192   

   52_Week_Low     SMA50    SMA200     EMA50  ...  Correlated_QQQX  \
1     0.080465  0.259052  0.000000  0.304612  ...         0.753202   
2     0.107192  0.266599  0.005536  0.312343  ...         0.726843   
3     0.107192  0.273794  0.010569  0.318728  ...         0.641437   
4     0.107192  0.279900  0.015253  0.322704  ...         0.588718   
5     0.107192  0.285177  0.019895  0.325206  ...         0.522291   

   Correlated_BST  Correlated_CSQ  Correlated_MSFT  Correlated_ETG  \
1        1.000000        1.000000         0.357467        0.794362   
2        0.996922        0.996880         0.378785        0.786317   
3        0.891744        0.931358         0.348979        0.748385   
4        0.864038        0.904837         0.351664        0.751835   
5        0.840950        0.864276         0.344834        0.701259   

   Correlated_ETY  Correlated_EXG  Future_Close_14    %Close  Future_%Close_14  
1        0.456706        0.997344         0.335457 -0.003535         -0.085375  
2        0.448452        1.000000         0.328369  0.025004         -0.112027  
3        0.420940        0.954848         0.311861 -0.012692         -0.110851  
4        0.403057        0.952192         0.311053 -0.026600         -0.087068  
5        0.375545        0.922976         0.306837 -0.016693         -0.074302  

[5 rows x 50 columns]

In [14]:
bin_edges = [-1, -0.5, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.5, 1]
df[f'Future_%Close_{n_days}_Binned'] = pd.cut(df[f'Future_%Close_{n_days}'], bins=bin_edges)
df_one_hot = pd.get_dummies(df[f'Future_%Close_{n_days}_Binned'], prefix=f'Future_%Close_{n_days}_bin')
df = pd.concat([df, df_one_hot], axis=1)
df.head()

Date     Close      High       Low    Volume  52_Week_High  \
1  2021-12-31  0.478598  0.470126  0.487422  0.135810      0.080465   
2  2022-01-03  0.519035  0.503472  0.491554  0.273018      0.107192   
3  2022-01-04  0.497996  0.504020  0.504499  0.255445      0.107192   
4  2022-01-05  0.454463  0.478714  0.463368  0.239246      0.107192   
5  2022-01-06  0.427869  0.434223  0.435824  0.247277      0.107192   

   52_Week_Low     SMA50    SMA200     EMA50  ...  \
1     0.080465  0.259052  0.000000  0.304612  ...   
2     0.107192  0.266599  0.005536  0.312343  ...   
3     0.107192  0.273794  0.010569  0.318728  ...   
4     0.107192  0.279900  0.015253  0.322704  ...   
5     0.107192  0.285177  0.019895  0.325206  ...   

   Future_%Close_14_bin_(-1.0, -0.5]  Future_%Close_14_bin_(-0.5, -0.3]  \
1                              False                              False   
2                              False                              False   
3                              False                              False   
4                              False                              False   
5                              False                              False   

   Future_%Close_14_bin_(-0.3, -0.2]  Future_%Close_14_bin_(-0.2, -0.1]  \
1                              False                              False   
2                              False                               True   
3                              False                               True   
4                              False                              False   
5                              False                              False   

   Future_%Close_14_bin_(-0.1, 0.0]  Future_%Close_14_bin_(0.0, 0.1]  \
1                              True                            False   
2                             False                            False   
3                             False                            False   
4                              True                            False   
5                              True                            False   

   Future_%Close_14_bin_(0.1, 0.2]  Future_%Close_14_bin_(0.2, 0.3]  \
1                            False                            False   
2                            False                            False   
3                            False                            False   
4                            False                            False   
5                            False                            False   

   Future_%Close_14_bin_(0.3, 0.5]  Future_%Close_14_bin_(0.5, 1.0]  
1                            False                            False  
2                            False                            False  
3                            False                            False  
4                            False                            False  
5                            False                            False  

[5 rows x 61 columns]